This file is my implementation of ehsan's code for testing model A on the images generated by the same model (FGSM)

In [ ]:
# Running not on full dataset: train_samples=2929 test_samples=568
# Put this in a Jupyter cell
from __future__ import annotations
import os
import random
from pathlib import Path
from typing import List, Tuple, Dict
from cornet import cornet_s
import sys
print(sys.version)

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
import torchvision.models as models

# -------------------------
# Dataset / splitting (same deterministic split used previously)
# -------------------------
class FewPerClassImageDataset(Dataset):
    """Simple dataset wrapping lists of (image_path, label)."""
    def __init__(self, samples: List[Tuple[str, int]], transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        p, label = self.samples[idx]
        img = Image.open(p).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, label

def build_splits_from_folder(root: str, train_per_class: int = 3, test_per_class: int = 1,
                             shuffle_within_class: bool = False) -> Tuple[List[Tuple[str,int]], List[Tuple[str,int]], Dict[int,str]]:
    """
    Walks `root` and for each subfolder (class) picks the first `train_per_class` images
    as train and next `test_per_class` images as test. Returns lists of (path, label).
    Also returns a label -> class_name map.
    """
    root_p = Path(root)
    assert root_p.exists(), f"Data root not found: {root}"
    classes = sorted([d for d in root_p.iterdir() if d.is_dir()])
    train_samples = []
    test_samples = []
    class_map = {}
    for label, cls in enumerate(classes):
        imgs = sorted([p for p in cls.iterdir() if p.is_file()])
        if shuffle_within_class:
            random.shuffle(imgs)
        # Basic safety: slice to available images
        tr = imgs[:train_per_class]
        te = imgs[train_per_class:train_per_class + test_per_class]
        for p in tr:
            train_samples.append((str(p), label))
        for p in te:
            test_samples.append((str(p), label))
        class_map[label] = cls.name
    return train_samples, test_samples, class_map

# -------------------------
# FGSM helpers
# -------------------------
def fgsm_perturb_from_grad(x: torch.Tensor, grad: torch.Tensor, epsilon: float) -> torch.Tensor:
    """Perturb in normalized input space using gradient sign."""
    return torch.clamp(x + epsilon * grad.sign(), -10.0, 10.0).detach()

# -------------------------
# Evaluation functions
# -------------------------
def evaluate_clean(model: torch.nn.Module, loader: DataLoader, device: torch.device) -> float:
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return (correct / total) if total > 0 else 0.0

def evaluate_fgsm(model: torch.nn.Module, loader: DataLoader, device: torch.device,
                  epsilons: List[float], loss_fn=None) -> Dict[float, float]:
    """
    Runs FGSM on the loader (same validation set). Returns mapping epsilon -> adversarial accuracy.
    Uses one backward per batch and re-uses gradient sign for efficiency.
    """
    if loss_fn is None:
        loss_fn = nn.CrossEntropyLoss()
    model.eval()
    results = {eps: {'correct': 0, 'total': 0} for eps in epsilons}

    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        # Need gradients w.r.t. inputs
        x.requires_grad = True
        out = model(x)
        loss = loss_fn(out, y)
        model.zero_grad()
        loss.backward()
        grad = x.grad.data  # gradient of loss wrt normalized input
        for eps in epsilons:
            x_adv = fgsm_perturb_from_grad(x, grad, eps)
            with torch.no_grad():
                out_adv = model(x_adv)
                preds = out_adv.argmax(dim=1)
                results[eps]['correct'] += (preds == y).sum().item()
                results[eps]['total'] += y.size(0)
        # clear grad for next batch
        x.grad = None

    accs = {eps: (results[eps]['correct'] / results[eps]['total'] if results[eps]['total'] > 0 else 0.0)
            for eps in epsilons}
    return accs

# -------------------------
# Model helpers (pretrained)
# -------------------------
def load_pretrained_alexnet(num_classes_expected:int, device:torch.device):
    model = models.alexnet(pretrained=True)  # pretrained weights
    # If dataset classes != 1000 then user is evaluating a different label space.
    # We'll keep pretrained classifier only if num_classes_expected == 1000.
    print(num_classes_expected)
    if num_classes_expected != 1000:
        print(f"[warning] dataset has {num_classes_expected} classes != 1000. Replacing AlexNet final classifier (random init).")
        in_feats = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_feats, num_classes_expected)
    return model.to(device)

def load_pretrained_vgg16(num_classes_expected:int, device:torch.device):
    model = models.vgg16(pretrained=True)
    if num_classes_expected != 1000:
        print(f"[warning] dataset has {num_classes_expected} classes != 1000. Replacing VGG16 final classifier (random init).")
        in_feats = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_feats, num_classes_expected)
    return model.to(device)

def load_pretrained_cornet(num_classes_expected: int, device: torch.device):
    # Load CORnet-S (ImageNet-pretrained)
    model = cornet_s(pretrained=True)

    # CORnet-S classifier is `decoder`
    if num_classes_expected != 1000:
        in_feats = model.decoder.in_features
        model.decoder = nn.Linear(in_feats, num_classes_expected)
        print(
            f"[warning] CORnet-S: dataset has {num_classes_expected} classes != 1000; "
            "final layer replaced (random init)."
        )
    return model.to(device)
# -------------------------
# Main evaluation runner (no training)
# -------------------------
def eval_pretrained_models(
    data_dir: str,
    train_per_class: int = 3,
    test_per_class: int = 1,
    batch_size: int = 128,
    num_workers: int = 4,
    device: str | None = None,
    models_to_run: List[str] = ("alexnet", "vgg16", "cornet"),
    epsilons: List[float] = (0.1, 0.01, 0.001),
):
    """
    Load pretrained AlexNet & VGG16 and evaluate on the validation/test split (no training).
    Also run FGSM on the SAME validation set.

    Example:
      eval_pretrained_models("/path/to/root", batch_size=64, device="cuda:0")
    """
    device = torch.device(device if device else ("cuda" if torch.cuda.is_available() else "cpu"))
    print(f"[eval_pretrained_models] using device: {device}")

    train_samples, test_samples, class_map = build_splits_from_folder(data_dir, train_per_class=train_per_class, test_per_class=test_per_class)
    n_classes = len(class_map)
    print(f"Found {n_classes} classes. train_samples={len(train_samples)} test_samples={len(test_samples)}")

    # Deterministic transforms: 224x224, same for train & test (we only evaluate on test here)
    input_size = 224
    test_transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    ])

    test_ds = FewPerClassImageDataset(test_samples, transform=test_transform)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    results = {}
    for model_name in models_to_run:
        model_name = model_name.lower()
        if model_name == "alexnet":
            model = load_pretrained_alexnet(n_classes, device)
        elif model_name == "vgg16":
            model = load_pretrained_vgg16(n_classes, device)
        elif model_name == "cornet" or "cornet_s":
            model = load_pretrained_cornet(n_classes, device)
        else:
            print(f"[eval_pretrained_models] unknown model '{model_name}' - skipping")
            continue

        # Evaluate clean accuracy on validation set
        clean_acc = evaluate_clean(model, test_loader, device)
        print(f"\n[{model_name}] clean accuracy on validation set: {clean_acc:.4f}")

        # Evaluate FGSM adversarial accuracies on same validation set
        adv_accs = evaluate_fgsm(model, test_loader, device, list(epsilons), loss_fn=nn.CrossEntropyLoss())
        for eps, acc in adv_accs.items():
            print(f"[{model_name}] FGSM eps={eps:.4g} adversarial accuracy: {acc:.4f}; acc percent: {acc*100:.2f}%")

        results[model_name] = {"clean_acc": clean_acc, "adv_accs": adv_accs}

    return results

# -------------------------
# Example invocation (edit path and run in cell)
# -------------------------
# res = eval_pretrained_models("/path/to/root", batch_size=64, device=None)
# print(res)

res = eval_pretrained_models(
    "val/val",
    batch_size=16,
    device=None,
    models_to_run=("alexnet", "vgg16", "cornet"))
print(res)

3.10.19 (main, Oct 21 2025, 16:43:05) [GCC 11.2.0]
[eval_pretrained_models] using device: cuda
Found 1000 classes. train_samples=2929 test_samples=568


/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1000

[alexnet] clean accuracy on validation set: 0.5792
[alexnet] FGSM eps=0.1 adversarial accuracy: 0.0053; acc percent: 0.53%
[alexnet] FGSM eps=0.01 adversarial accuracy: 0.1690; acc percent: 16.90%
[alexnet] FGSM eps=0.001 adversarial accuracy: 0.5106; acc percent: 51.06%


/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



[vgg16] clean accuracy on validation set: 0.7077
[vgg16] FGSM eps=0.1 adversarial accuracy: 0.0511; acc percent: 5.11%
[vgg16] FGSM eps=0.01 adversarial accuracy: 0.1373; acc percent: 13.73%
[vgg16] FGSM eps=0.001 adversarial accuracy: 0.5845; acc percent: 58.45%

[cornet] clean accuracy on validation set: 0.7025
[cornet] FGSM eps=0.1 adversarial accuracy: 0.0387; acc percent: 3.87%
[cornet] FGSM eps=0.01 adversarial accuracy: 0.1673; acc percent: 16.73%
[cornet] FGSM eps=0.001 adversarial accuracy: 0.6127; acc percent: 61.27%
{'alexnet': {'clean_acc': 0.579225352112676, 'adv_accs': {0.1: 0.00528169014084507, 0.01: 0.16901408450704225, 0.001: 0.5105633802816901}}, 'vgg16': {'clean_acc': 0.7077464788732394, 'adv_accs': {0.1: 0.051056338028169015, 0.01: 0.13732394366197184, 0.001: 0.5845070422535211}}, 'cornet': {'clean_acc': 0.7024647887323944, 'adv_accs': {0.1: 0.03873239436619718, 0.01: 0.16725352112676056, 0.001: 0.6126760563380281}}}


In [ ]:
# Running on full dataset: 3923 total images
# Put this into a Jupyter notebook cell
from __future__ import annotations
import os
from cornet import cornet_s
from pathlib import Path
from typing import List, Tuple, Dict

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
import torchvision.models as models

# -------------------------
# Dataset: ALL images = test set
# -------------------------
class AllImagesAsTestDataset(Dataset):
    """Wraps a list of (image_path, label) - used as the entire test set (no split)."""
    def __init__(self, samples: List[Tuple[str, int]], transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        p, label = self.samples[idx]
        img = Image.open(p).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, label

def build_all_test_samples(root: str, sort_filenames: bool = True) -> Tuple[List[Tuple[str,int]], Dict[int, str]]:
    """
    Walks `root` and builds a list of (image_path, label) for *all* files in each class folder.
    Label assigned by sorted folder order. Returns samples and class_map.
    """
    root_p = Path(root)
    assert root_p.exists(), f"Data root not found: {root}"
    class_dirs = sorted([d for d in root_p.iterdir() if d.is_dir()])
    samples = []
    class_map = {}
    for label, cls in enumerate(class_dirs):
        imgs = [p for p in cls.iterdir() if p.is_file()]
        if sort_filenames:
            imgs = sorted(imgs)
        for p in imgs:
            samples.append((str(p), label))
        class_map[label] = cls.name
    return samples, class_map

# -------------------------
# FGSM helper (normalized-space)
# -------------------------
def fgsm_perturb_from_grad(x: torch.Tensor, grad: torch.Tensor, epsilon: float) -> torch.Tensor:
    """Return perturbed inputs in normalized input space using gradient sign."""
    return torch.clamp(x + epsilon * grad.sign(), -10.0, 10.0).detach()

# -------------------------
# Evaluation (clean + FGSM)
# -------------------------
def evaluate_clean(model: nn.Module, loader: DataLoader, device: torch.device) -> float:
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return (correct / total) if total > 0 else 0.0

def evaluate_fgsm(model: nn.Module, loader: DataLoader, device: torch.device,
                  epsilons: List[float], loss_fn=None) -> Dict[float, float]:
    if loss_fn is None:
        loss_fn = nn.CrossEntropyLoss()
    model.eval()
    results = {eps: {'correct': 0, 'total': 0} for eps in epsilons}

    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        x.requires_grad = True
        out = model(x)
        loss = loss_fn(out, y)
        model.zero_grad()
        loss.backward()
        grad = x.grad.data
        for eps in epsilons:
            x_adv = fgsm_perturb_from_grad(x, grad, eps)
            with torch.no_grad():
                out_adv = model(x_adv)
                preds = out_adv.argmax(dim=1)
                results[eps]['correct'] += (preds == y).sum().item()
                results[eps]['total'] += y.size(0)
        x.grad = None

    return {eps: (results[eps]['correct'] / results[eps]['total'] if results[eps]['total'] > 0 else 0.0)
            for eps in epsilons}

# -------------------------
# Pretrained model loaders
# -------------------------
def load_pretrained_alexnet(num_classes_expected: int, device: torch.device):
    model = models.alexnet(pretrained=True)
    if num_classes_expected != 1000:
        # Replace final layer to match labels (user dataset labels)
        in_feats = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_feats, num_classes_expected)
        # note: the new head is random init (we're not training it here)
        print(f"[warning] AlexNet: dataset has {num_classes_expected} classes != 1000; final layer replaced (random init).")
    return model.to(device)

def load_pretrained_vgg16(num_classes_expected: int, device: torch.device):
    model = models.vgg16(pretrained=True)
    if num_classes_expected != 1000:
        in_feats = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_feats, num_classes_expected)
        print(f"[warning] VGG16: dataset has {num_classes_expected} classes != 1000; final layer replaced (random init).")
    return model.to(device)

def load_pretrained_cornet(num_classes_expected: int, device: torch.device):
    # Load CORnet-S (ImageNet-pretrained)
    model = cornet_s(pretrained=True)

    # CORnet-S classifier is `decoder`
    if num_classes_expected != 1000:
        in_feats = model.decoder.in_features
        model.decoder = nn.Linear(in_feats, num_classes_expected)
        print(
            f"[warning] CORnet-S: dataset has {num_classes_expected} classes != 1000; "
            "final layer replaced (random init)."
        )
    return model.to(device)
# -------------------------
# Main runner: ALL images are test set
# -------------------------
def eval_pretrained_on_all_test(
    data_dir: str,
    batch_size: int = 128,
    num_workers: int = 4,
    device: str | None = None,
    models_to_run: List[str] = ("alexnet", "vgg16", "cornet_s"),
    epsilons: List[float] = (0.005, 0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3, 0.5),
):
    """
    Evaluate pretrained AlexNet/VGG16 on ALL images in `data_dir` (root/class_x/*.jpg).
    All images are treated as test samples (no splitting, no training).
    Returns a dict with clean and adversarial accuracies per model.
    """
    device = torch.device(device if device else ("cuda" if torch.cuda.is_available() else "cpu"))
    print(f"[eval_pretrained_on_all_test] using device: {device}")

    samples, class_map = build_all_test_samples(data_dir)
    n_classes = len(class_map)
    print(f"Found {n_classes} classes and {len(samples)} total images (all used as test samples).")

    # Deterministic transform (224 x 224), same for all images
    input_size = 224
    transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    ])

    test_ds = AllImagesAsTestDataset(samples, transform=transform)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    results = {}
    for model_name in models_to_run:
        model_name = model_name.lower()
        if model_name == "alexnet":
            model = load_pretrained_alexnet(n_classes, device)
        elif model_name == "vgg16":
            model = load_pretrained_vgg16(n_classes, device)
        elif model_name in ("cornet", "cornet_s"):
            model = load_pretrained_cornet(n_classes, device)
        else:
            print(f"[eval_pretrained_on_all_test] unknown model '{model_name}' - skipping")
            continue

        print(f"\n--- Evaluating {model_name} on full dataset ---")
        clean_acc = evaluate_clean(model, test_loader, device)
        print(f"{model_name} clean accuracy: {clean_acc:.4f}")

        adv_accs = evaluate_fgsm(model, test_loader, device, list(epsilons), loss_fn=nn.CrossEntropyLoss())
        for eps, acc in adv_accs.items():
            print(f"{model_name} FGSM eps={eps:.4g} adv accuracy: {acc:.4f}; acc percent: {acc*100:.2f}%")

        results[model_name] = {"clean_acc": clean_acc, "adv_accs": adv_accs}

    print("[eval_pretrained_on_all_test] done.")
    return results

# -------------------------
# Example usage (edit path):
# -------------------------
# /imaging/mrahm326/val/
results = eval_pretrained_on_all_test(
    "val/val/", batch_size=16, device="cuda:0",
    models_to_run=("alexnet", "vgg16", "cornet"))
print(results)


[eval_pretrained_on_all_test] using device: cuda:0
Found 1000 classes and 3923 total images (all used as test samples).


/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



--- Evaluating alexnet on full dataset ---
alexnet clean accuracy: 0.5631
alexnet FGSM eps=0.005 adv accuracy: 0.3026; acc percent: 30.26%
alexnet FGSM eps=0.01 adv accuracy: 0.1637; acc percent: 16.37%
alexnet FGSM eps=0.02 adv accuracy: 0.0612; acc percent: 6.12%
alexnet FGSM eps=0.03 adv accuracy: 0.0303; acc percent: 3.03%
alexnet FGSM eps=0.05 adv accuracy: 0.0112; acc percent: 1.12%
alexnet FGSM eps=0.1 adv accuracy: 0.0054; acc percent: 0.54%
alexnet FGSM eps=0.2 adv accuracy: 0.0046; acc percent: 0.46%
alexnet FGSM eps=0.3 adv accuracy: 0.0028; acc percent: 0.28%
alexnet FGSM eps=0.5 adv accuracy: 0.0020; acc percent: 0.20%


/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



--- Evaluating vgg16 on full dataset ---
vgg16 clean accuracy: 0.7043
vgg16 FGSM eps=0.005 adv accuracy: 0.2796; acc percent: 27.96%
vgg16 FGSM eps=0.01 adv accuracy: 0.1318; acc percent: 13.18%
vgg16 FGSM eps=0.02 adv accuracy: 0.0551; acc percent: 5.51%
vgg16 FGSM eps=0.03 adv accuracy: 0.0367; acc percent: 3.67%
vgg16 FGSM eps=0.05 adv accuracy: 0.0296; acc percent: 2.96%
vgg16 FGSM eps=0.1 adv accuracy: 0.0298; acc percent: 2.98%
vgg16 FGSM eps=0.2 adv accuracy: 0.0362; acc percent: 3.62%
vgg16 FGSM eps=0.3 adv accuracy: 0.0367; acc percent: 3.67%
vgg16 FGSM eps=0.5 adv accuracy: 0.0275; acc percent: 2.75%

--- Evaluating cornet on full dataset ---
cornet clean accuracy: 0.7214
cornet FGSM eps=0.005 adv accuracy: 0.3569; acc percent: 35.69%
cornet FGSM eps=0.01 adv accuracy: 0.1960; acc percent: 19.60%
cornet FGSM eps=0.02 adv accuracy: 0.0795; acc percent: 7.95%
cornet FGSM eps=0.03 adv accuracy: 0.0510; acc percent: 5.10%
cornet FGSM eps=0.05 adv accuracy: 0.0339; acc percent: 3

BEFORE CHANGING EPSILONS TO MY RANGE OF EPSILONS
--- Evaluating alexnet on full dataset ---
alexnet clean accuracy: 0.5631
alexnet FGSM eps=0.1 adv accuracy: 0.0054; acc percent: 0.54%
alexnet FGSM eps=0.01 adv accuracy: 0.1637; acc percent: 16.37%
alexnet FGSM eps=0.005 adv accuracy: 0.3026; acc percent: 30.26%
alexnet FGSM eps=0.001 adv accuracy: 0.4986; acc percent: 49.86%
/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

--- Evaluating vgg16 on full dataset ---
vgg16 clean accuracy: 0.7043
vgg16 FGSM eps=0.1 adv accuracy: 0.0298; acc percent: 2.98%
vgg16 FGSM eps=0.01 adv accuracy: 0.1318; acc percent: 13.18%
vgg16 FGSM eps=0.005 adv accuracy: 0.2796; acc percent: 27.96%
vgg16 FGSM eps=0.001 adv accuracy: 0.5927; acc percent: 59.27%

--- Evaluating cornet on full dataset ---
cornet clean accuracy: 0.7214
cornet FGSM eps=0.1 adv accuracy: 0.0298; acc percent: 2.98%
cornet FGSM eps=0.01 adv accuracy: 0.1960; acc percent: 19.60%
cornet FGSM eps=0.005 adv accuracy: 0.3569; acc percent: 35.69%
cornet FGSM eps=0.001 adv accuracy: 0.6289; acc percent: 62.89%